In [0]:
create streaming table sales_ash as 
select *,current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/sales',
  format => 'csv'
)



In [0]:
create streaming table product_ash as 
select *,current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/products',
  format => 'csv'
)

In [0]:
create streaming table customer_ash as 
select *,current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/customers',
  format => 'csv'
)

In [0]:
create streaming table ashwin_silver.sales_silver  
(CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW) as
select distinct * except(_rescued_data,ingestion_date) from stream ashwin_bronze.sales_ash

In [0]:
CREATE OR REFRESH STREAMING TABLE ashwin_silver.product_ash_silver;

APPLY CHANGES INTO
  ashwin_silver.product_ash_silver
FROM
  stream(ashwin_bronze.product_ash)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum,_rescued_data,ingestion_date)
STORED AS
  SCD TYPE 1;

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE ashwin_silver.customer_ash_silver;

APPLY CHANGES INTO
  ashwin_silver.customer_ash_silver
FROM
  stream(naval_bronze.customer_ash)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum,_rescued_data,ingestion_date)
STORED AS
  SCD TYPE 2;

In [0]:
create materialized view ashwin_gold.customer_active as 
select * from ashwin_silver.customer_ash_silver where `__END_AT` is null